# "THE PRICE IS RIGHT" Capstone Project

This week - build a model that predicts how much something costs from a description, based on a scrape of Amazon data


A model that can estimate how much something costs, from its description.

# Order of play

DAY 1: Data Curation  
DAY 2: Data Pre-processing  
DAY 3: Evaluation, Baselines, Traditional ML  
DAY 4: Deep Learning and LLMs  
DAY 5: Fine-tuning a Frontier Model  

## DAY 4: Neural Networks and LLMs

Today we'll work from Traditional ML to Neural Networks to Large Language Models!!

In [1]:
# imports

import os
from dotenv import load_dotenv
from huggingface_hub import login
from pricer.evaluator import evaluate
from litellm import completion
from pricer.items import Item
import numpy as np
from tqdm.notebook import tqdm
import csv
from sklearn.feature_extraction.text import HashingVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR


In [2]:
LITE_MODE = True

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


# Before we look at the Artificial Neural Networks

## There is a different kind of Neural Network we could consider

In [4]:
# Write the test set to a CSV

with open('human_in.csv', 'w', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:100]:
        writer.writerow([t.summary, 0])

In [5]:
# Read it back in

human_predictions = []
with open('human_out.csv', 'r', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[1]))

In [6]:
def human_pricer(item):
    idx = test.index(item)
    return human_predictions[idx]

In [7]:
human = human_pricer(test[0])
actual = test[0].price
print(f"Human predicted {human} for an item that actually costs {actual}")


Human predicted 120.0 for an item that actually costs 219.0


In [8]:
evaluate(human_pricer, test, size=100)

  0%|          | 0/100 [00:00<?, ?it/s]

$99 $184 $12 $15 $18 $10 $119 $135 $6 $270 $643 $329 $15 $26 $24 $18 $29 $25 $25 $53 $35 $126 $25 $127 $273 $398 $55 $6 $101 $51 $30 $5 $35 $9 $10 $419 $25 $11 $186 $33 $161 $51 $23 $155 $150 $4 $31 $18 $115 $82 $25 $111 $410 $75 $67 $34 $8 $10 $122 $28 $116 $17 $19 $60 $599 $60 $160 $355 $75 $34 $17 $2 $70 $76 $41 $9 $226 $5 $5 $4 $0 $7 $5 $74 $7 $10 $68 $74 $5 $3 $17 $45 $5 $16 $0 $153 $2 $122 $150 $355 

# And now - a vanilla Neural Network

During the remainder of this course we will get deeper into how Neural Networks work, and how to train a neural network.

This is just a sneak preview - let's make our own Neural Network, from scratch, using Pytorch.

Use this to get intuition; it's not important to know all about Neural networks at this point..

In [9]:
# Prepare our documents and prices

y = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

In [10]:
# Use the HashingVectorizer for a Bag of Words model
# Using binary=True with the CountVectorizer makes "one-hot vectors"

np.random.seed(42)
vectorizer = HashingVectorizer(n_features=5000, stop_words='english', binary=True)
X = vectorizer.fit_transform(documents)

In [11]:
# Define the neural network - here is Pytorch code to create a 8 layer neural network

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 64)
        self.layer4 = nn.Linear(64, 64)
        self.layer5 = nn.Linear(64, 64)
        self.layer6 = nn.Linear(64, 64)
        self.layer7 = nn.Linear(64, 64)
        self.layer8 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        output1 = self.relu(self.layer1(x))
        output2 = self.relu(self.layer2(output1))
        output3 = self.relu(self.layer3(output2))
        output4 = self.relu(self.layer4(output3))
        output5 = self.relu(self.layer5(output4))
        output6 = self.relu(self.layer6(output5))
        output7 = self.relu(self.layer7(output6))
        output8 = self.layer8(output7)
        return output8

In [12]:
# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X.toarray())
y_train_tensor = torch.FloatTensor(y).unsqueeze(1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.01, random_state=42)

# Create the loader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

In [13]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {trainable_params:,}")

Number of trainable parameters: 669,249


In [14]:
# Define loss function and optimizer

loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# We will do 2 complete runs through the data

EPOCHS = 2

for epoch in range(EPOCHS):
    model.train()
    for batch_X, batch_y in tqdm(train_loader):
        optimizer.zero_grad()

        # The next 4 lines are the 4 stages of training: forward pass, loss calculation, backward pass, optimize
        
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = loss_function(val_outputs, y_val)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {loss.item():.3f}, Val Loss: {val_loss.item():.3f}')

  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [1/2], Train Loss: 16197.763, Val Loss: 20532.570


  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [2/2], Train Loss: 7266.796, Val Loss: 17743.707


In [15]:
def neural_network(item):
    model.eval()
    with torch.no_grad():
        vector = vectorizer.transform([item.summary])
        vector = torch.FloatTensor(vector.toarray())
        result = model(vector)[0].item()
    return max(0, result)

In [16]:
evaluate(neural_network, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$69 $58 $16 $19 $37 $167 $13 $83 $33 $26 $478 $153 $74 $144 $17 $8 $7 $26 $60 $31 $34 $41 $74 $3 $271 $287 $246 $24 $36 $38 $42 $149 $22 $21 $123 $308 $42 $115 $121 $59 $164 $105 $20 $54 $120 $46 $79 $60 $23 $35 $17 $38 $100 $25 $90 $57 $39 $122 $12 $35 $31 $11 $37 $3 $419 $78 $6 $260 $13 $261 $12 $30 $121 $122 $14 $48 $44 $38 $27 $53 $65 $123 $38 $41 $7 $48 $56 $115 $98 $143 $24 $118 $11 $14 $33 $75 $40 $32 $132 $210 $30 $57 $3 $42 $1 $34 $116 $226 $8 $105 $34 $97 $121 $54 $30 $130 $134 $61 $54 $56 $23 $164 $37 $19 $77 $18 $25 $182 $52 $47 $46 $105 $113 $17 $73 $24 $113 $51 $40 $71 $55 $159 $11 $141 $205 $74 $75 $301 $60 $12 $19 $227 $12 $62 $39 $137 $145 $12 $11 $16 $78 $3 $2 $20 $518 $17 $62 $23 $24 $44 $15 $20 $234 $57 $32 $8 $15 $2 $36 $119 $377 $15 $79 $32 $38 $123 $46 $33 $25 $11 $61 $56 $66 $4 $9 $22 $58 $30 $1 $19 

# And now - to the frontier!

Let's see how Frontier models do out of the box; no training, just inference based on their world knowledge.

Tomorrow we will do some training.

In [4]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [{"role": "user", "content": message}]

In [5]:
print(test[0].summary)

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.


In [6]:
messages_for(test[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Excess V2 Distortion/Modulation Pedal  \nCategory: Music Pedals  \nBrand: Old Blood Noise  \nDescription: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  \nDetails: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.'}]

In [20]:
# The function for gpt-4.1-nano

def gpt_4__1_nano(item):
    response = completion(model="openai/gpt-4.1-nano", messages=messages_for(item))
    return response.choices[0].message.content

In [21]:
gpt_4__1_nano(test[0])

'$200'

In [22]:
test[0].price

219.0

In [23]:
evaluate(gpt_4__1_nano, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$39 $34 $30 $5 $120 $80 $6 $65 $1 $870 $263 $20 $30 $19 $19 $8 $71 $0 $10 $31 $64 $76 $35 $75 $182 $303 $405 $5 $201 $64 $50 $15 $40 $40 $35 $169 $90 $26 $36 $18 $175 $35 $20 $105 $70 $5 $27 $13 $85 $52 $22 $105 $125 $10 $247 $14 $8 $59 $52 $13 $86 $28 $41 $30 $29 $60 $90 $295 $25 $74 $16 $8 $130 $4 $14 $11 $126 $0 $8 $3 $30 $3 $15 $74 $14 $20 $32 $56 $30 $21 $3 $25 $5 $20 $4 $78 $1 $7 $70 $425 $50 $3 $11 $11 $1 $132 $10 $350 $19 $49 $20 $186 $49 $48 $54 $80 $5 $5 $24 $47 $14 $261 $80 $16 $50 $10 $10 $151 $29 $89 $179 $13 $15 $5 $35 $0 $55 $0 $53 $42 $16 $150 $40 $14 $134 $118 $20 $440 $15 $13 $3 $144 $3 $3360 $3 $29 $71 $41 $30 $5 $311 $17 $3 $2 $240 $3 $752 $25 $5 $5 $5 $3 $120 $2 $52 $101 $3 $57 $26 $43 $146 $15 $150 $1 $30 $3 $83 $13 $20 $2 $25 $99 $5 $11 $20 $70 $10 $70 $11 $1 

In [24]:
def claude_opus_4_5(item):
    response = completion(model="anthropic/claude-opus-4-5", messages=messages_for(item))
    return response.choices[0].message.content

In [28]:
evaluate(claude_opus_4_5, test, workers=1, size=100)

  0%|          | 0/100 [00:00<?, ?it/s]

$20 $34 $25 $25 $0 $80 $54 $69 $6 $45 $214 $129 $5 $21 $49 $3 $11 $20 $20 $74 $44 $1 $40 $55 $33 $254 $206 $5 $90 $64 $10 $30 $70 $50 $25 $369 $14 $43 $34 $8 $154 $55 $10 $5 $70 $1 $5 $2 $65 $72 $28 $114 $345 $10 $37 $44 $6 $50 $48 $1 $106 $48 $61 $60 $279 $9 $50 $355 $35 $14 $19 $3 $20 $6 $20 $16 $25 $2 $2 $6 $30 $3 $5 $74 $14 $25 $32 $44 $30 $0 $3 $5 $4 $22 $1 $78 $4 $67 $120 $225 

In [29]:
def gemini_3_pro_preview(item):
    response = completion(model="gemini/gemini-3-pro-preview", messages=messages_for(item), reasoning_effort='low')
    return response.choices[0].message.content

In [30]:
evaluate(gemini_3_pro_preview, test, size=50, workers=2)

  0%|          | 0/50 [00:00<?, ?it/s]

$10 $84 $5 $25 $10 $45 $74 $80 $13 $160 $222 $150 $5 $4 $49 $4 $16 $13 $70 $20 $101 $19 $20 $5 $27 $184 $136 $5 $141 $60 $3 $30 $40 $44 $5 $150 $15 $39 $16 $8 $130 $31 $12 $95 $0 $1 $5 $2 $75 $48 

In [ ]:
def gemini_2__5_flash_lite(item):
    response = completion(model="gemini/gemini-2.5-flash-lite", messages=messages_for(item))
    return response.choices[0].message.content

In [34]:
evaluate(gemini_2__5_flash_lite, test, size=195)

  0%|          | 0/195 [00:00<?, ?it/s]

$21 $64 $15 $60 $50 $80 $59 $65 $1 $120 $313 $79 $15 $9 $29 $12 $71 $15 $240 $54 $16 $9 $10 $25 $112 $233 $145 $5 $351 $62 $15 $15 $10 $55 $35 $31 $15 $41 $64 $13 $155 $35 $15 $95 $70 $5 $7 $13 $70 $22 $29 $105 $125 $10 $67 $16 $8 $35 $82 $13 $36 $23 $61 $20 $129 $30 $60 $260 $0 $144 $13 $8 $30 $4 $15 $11 $26 $0 $8 $4 $30 $3 $5 $74 $8 $20 $168 $21 $30 $11 $3 $30 $15 $10 $0 $78 $1 $37 $95 $225 $50 $33 $8 $61 $51 $82 $17 $350 $14 $29 $15 $136 $44 $43 $4 $80 $0 $1 $64 $347 $19 $161 $30 $16 $0 $15 $5 $51 $71 $74 $54 $13 $10 $5 $210 $5 $85 $10 $22 $2 $26 $149 $5 $4 $56 $4 $20 $240 $85 $8 $1 $144 $12 $40 $4 $71 $46 $41 $20 $5 $110 $13 $23 $2 $90 $12 $1402 $15 $6 $5 $0 $3 $170 $8 $57 $101 $3 $3 $66 $28 $4 $20 $250 $1 $25 $13 $63 $17 $0 $2 $15 $24 $40 $81 $35 

In [6]:

def grok_4__1_fast(item):
    response = completion(model="xai/grok-4-1-fast-non-reasoning", messages=messages_for(item), seed=42)
    return response.choices[0].message.content

In [12]:
evaluate(grok_4__1_fast, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$20 $134 $30 $59 $20 $80 $54 $95 $11 $80 $187 $21 $5 $4 $29 $8 $1 $15 $90 $69 $14 $4 $6 $175 $18 $103 $145 $5 $351 $65 $30 $10 $90 $65 $35 $369 $90 $31 $14 $13 $175 $35 $25 $105 $100 $0 $7 $3 $65 $48 $25 $117 $225 $20 $3 $16 $8 $40 $48 $3 $116 $48 $46 $40 $229 $40 $10 $325 $5 $174 $17 $8 $30 $4 $30 $16 $26 $0 $2 $6 $40 $1 $5 $79 $2 $10 $68 $56 $0 $21 $3 $15 $5 $20 $4 $78 $1 $37 $20 $325 $20 $3 $12 $10 $49 $82 $17 $350 $9 $51 $10 $64 $19 $48 $54 $230 $10 $5 $24 $147 $4 $511 $10 $86 $0 $10 $5 $31 $29 $59 $79 $13 $5 $5 $185 $0 $55 $10 $22 $18 $16 $249 $30 $17 $74 $18 $5 $290 $85 $8 $4 $144 $22 $60 $4 $29 $71 $41 $100 $25 $211 $17 $78 $2 $140 $2 $752 $25 $0 $2 $15 $7 $170 $2 $57 $1 $7 $27 $14 $57 $146 $15 $150 $1 $30 $3 $83 $17 $10 $2 $5 $1 $10 $161 $11 $40 $80 $130 $26 $4 

In [8]:
# The function for gpt-5.1

def gpt_5__1(item):
    response = completion(model="gpt-5.1", messages=messages_for(item), reasoning_effort='high', seed=42)
    return response.choices[0].message.content


In [9]:
evaluate(gpt_5__1, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$20 $84 $15 $10 $10 $145 $54 $75 $11 $10 $37 $30 $5 $1 $39 $3 $41 $20 $40 $29 $16 $4 $5 $25 $52 $233 $145 $2 $71 $63 $5 $30 $40 $55 $15 $169 $30 $36 $54 $18 $160 $45 $15 $95 $40 $5 $7 $2 $65 $48 $26 $105 $200 $0 $27 $24 $4 $79 $78 $3 $106 $48 $43 $50 $159 $1 $60 $315 $15 $24 $16 $3 $80 $4 $22 $16 $56 $2 $1 $7 $0 $4 $10 $74 $12 $25 $68 $26 $0 $21 $3 $5 $5 $5 $2 $93 $1 $72 $50 $175 $0 $17 $2 $30 $29 $132 $14 $335 $1 $140 $20 $186 $9 $68 $54 $0 $7 $0 $24 $397 $4 $110 $10 $36 $0 $15 $0 $31 $11 $59 $129 $17 $5 $0 $25 $2 $25 $10 $122 $8 $1 $149 $15 $9 $6 $8 $15 $60 $5 $8 $3 $94 $27 $60 $4 $171 $41 $41 $50 $0 $110 $17 $23 $0 $41 $2 $451 $20 $0 $2 $10 $2 $170 $10 $67 $29 $4 $37 $4 $13 $105 $15 $250 $49 $20 $8 $63 $17 $15 $7 $5 $29 $10 $31 $60 $40 $9 $10 $16 $8 

## Open source models

In [22]:

def llama_3__2(item):
    response = completion(model="ollama/llama3.2", messages=messages_for(item), seed=42)
    return response.choices[0].message.content

In [23]:
evaluate(llama_3__2, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$1 $31 $31 $69 $70 $187 $34 $170 $14 $565 $318 $320 $30 $4 $20 $3 $41 $27 $90 $20 $25 $26 $38 $55 $132 $254 $355 $25 $149 $30 $5 $35 $239 $57 $24 $369 $30 $9 $85 $14 $160 $67 $26 $165 $110 $0 $2 $8 $65 $142 $11 $105 $325 $30 $646 $64 $13 $115 $62 $18 $263 $32 $46 $10 $469 $9 $10 $246 $35 $33 $17 $23 $150 $11 $43 $24 $775 $2 $23 $26 $30 $12 $30 $74 $23 $30 $42 $107 $30 $11 $18 $44 $8 $25 $6 $1 $36 $67 $201 $435 $20 $74 $7 $160 $119 $331 $0 $300 $23 $150 $0 $581 $198 $63 $6 $1130 $12 $11 $44 $137 $44 $411 $30 $126 $55 $35 $12 $29 $66 $69 $224 $113 $15 $5 $45 $5 $115 $30 $68 $41 $14 $179 $40 $40 $6 $8 $15 $90 $85 $17 $6 $54 $22 $5 $19 $199 $6 $44 $121 $40 $140 $20 $32 $12 $241 $3 $99 $40 $5 $0 $17 $3 $75 $27 $43 $1151 $2 $157 $14 $78 $346 $15 $95 $20 $375 $3 $53 $33 $0 $15 $5 $29 $10 $291 $10 $419 $30 $119 $21 $14 

In [24]:
def gpt_oss__20b(item):
    response = completion(model="ollama/gpt-oss:20b", messages=messages_for(item), seed=42)
    return response.choices[0].message.content

In [25]:
evaluate(gpt_oss__20b, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$31 $24 $15 $15 $40 $20 $36 $45 $22 $10 $433 $70 $29 $6 $9 $3 $1 $30 $75 $81 $106 $56 $50 $425 $222 $234 $455 $9 $81 $50 $120 $65 $190 $35 $25 $69 $20 $28 $36 $13 $165 $45 $25 $217 $175 $7 $27 $38 $10 $52 $21 $110 $5 $10 $497 $74 $21 $130 $88 $3 $114 $62 $51 $25 $489 $50 $141 $195 $55 $204 $17 $13 $55 $10 $18 $39 $226 $15 $23 $11 $60 $1 $80 $74 $5 $90 $167 $106 $60 $26 $3 $65 $2 $16 $0 $63 $14 $27 $232 $325 $79 $23 $18 $41 $69 $402 $0 $355 $9 $39 $40 $1086 $249 $147 $64 $480 $11 $1 $394 $247 $22 $561 $59 $26 $25 $15 $2 $281 $11 $104 $179 $243 $95 $0 $15 $4 $110 $109 $58 $72 $136 $0 $25 $2 $149 $198 $27 $340 $15 $3 $7 $119 $27 $210 $3 $41 $426 $43 $10 $20 $61 $13 $4 $5 $10 $17 $1352 $8 $18 $5 $1 $4 $70 $7 $158 $31 $32 $3 $44 $28 $196 $5 $150 $26 $850 $8 $58 $27 $20 $15 $15 $44 $0 $81 $79 $149 $10 $70 $16 $4 

In [11]:
def messages_for_to_usd(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation. Convert to USD.\n\n{item.summary}"
    return [{"role": "user", "content": message}]

def local__model(item):
    response = completion(model="ollama/deepseek-r1:1.5b", messages=messages_for_to_usd(item), seed=42)
    return response.choices[0].message.content

In [12]:
evaluate(local__model, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$169 $86 $25 $10 $5 $230 $118 $124 $9 $14670 $463 $309 $180 $196 $196 $117 $371 $35 $40 $54 $216 $126 $72 $25 $82 $333 $245 $6 $151 $15 $10 $20 $132 $76 $335 $369 $60 $39 $161 $7 $155 $65 $65 $180 $170 $21306 $12 $2 $70 $167 $29 $115 $405 $105 $322 $59 $28 $0 $182 $438 $166 $28 $91 $40 $499 $110 $1772 $195 $100 $124 $18 $253 $185 $14 $44 $144 $266 $75 $13 $5981 $0 $88 $120 $64 $12 $220 $182 $206 $30 $24 $3 $42 $580 $144 $22 $43 $51 $1154 $200 $15903 $38 $90 $34 $89 $115 $132 $22 $105 $69 $2712 $46 $189 $269 $37 $46 $9330 $65 $5 $1244 $1147 $84 $14 $10 $184 $182 $53 $20 $74 $271 $59 $516 $98 $64 $28 $85 $11 $165 $180 $58 $162 $29 $25 $2356 $44 $149 $18 $60 $110 $85 $167 $4 $393 $46 $115 $1 $224 $451 $44 $308 $35 $139 $18 $5 $10 $827 $97 $52 $15 $39 $25 $40 $7 $345 $137 $198 $26 $147 $58 $59 $65 $630 $0 $300 $224 $20 $19 $105 $17 $290 $7 $50 $149 $65 $139 $75 $40 $130 $30 $51 $11 